In [10]:
using DataStructures
using Distributions
using StableRNGs
using Printf
using Dates

## Entity data structure for each customer

mutable struct Lawnmower
    id::Int64
    arrival_time::Float64       
    start_service_time::Float64 
    completion_time::Float64    
end
# generate a newly arrived lawnmower 
Lawnmower(id, arrival_time) = Lawnmower(id, arrival_time, Inf, Inf)

### Events

abstract type Event end 

struct Arrival <: Event # lawnmower arrives
    id::Int64         # a unique event id
    time::Float64     # the time of the event 
end

mutable struct Departure <: Event # a lawnmower finsishes at machine i
    id::Int64         # a unique event id
    time::Float64     # the time of the event
end

mutable struct Breakdown_start <: Event
    id::Int64
    time::Float64  # additional service time due to breakdown
end

mutable struct Breakdown_end<: Event
    id::Int64
    time::Float64  # additional service time due to breakdown
end

struct Null <: Event 
    id::Int64    
end

### parameter structure
struct Parameters
    seed::Int
    T::Float64
    mean_interarrival_time::Float64
    mean_service_time::Float64
    mean_breakdown_interval::Float64
    mean_repair_time::Float64
    time_units::String
end

### function to write parameters. output.  IO

function write_parameters( output::IO, P::Parameters ) # function to writeout parameters
    T = typeof(P)
    for name in fieldnames(T)
        println( output, "# parameter: $name = $(getfield(P,name))" )
    end
end
write_parameters( P::Parameters ) = write_parameters( stdout, P )
function write_metadata( output::IO ) # function to writeout extra metadata
    (path, prog) = splitdir( @__FILE__ )
    println( output, "# file created by code in $(prog)" )
    t = now()
    println( output, "# file created on $(Dates.format(t, "yyyy-mm-dd at HH:MM:SS"))" )
end


### State

mutable struct SystemState
    time::Float64
    event_queue::PriorityQueue{Event, Float64}
    lawnmower_queue::Queue{Lawnmower}
    in_service::PriorityQueue{Lawnmower, Float64}
    n_entities::Int64
    n_events::Int64
    machine_status::Int
    current_repair_time::Float64
end

# initial empty space

function SystemState(P::Parameters)
    init_time = 0.0
    init_event_queue = PriorityQueue{Event, Float64}()
    init_lawnmower_queue = Queue{Lawnmower}()
    init_in_service = PriorityQueue{Lawnmower, Float64}()
    init_n_entities = 0
    init_n_events = 0
    machine_status = 0              # 0 means machine working , 1 means it broken         
    current_repair_time = 0.0

    # Make sure the Lawnmower of arguments matches the struct definition
    return SystemState(
        init_time,
        init_event_queue,
        init_lawnmower_queue,
        init_in_service,
        init_n_entities,
        init_n_events, 
        machine_status,
        current_repair_time
    )
end

# setup random number generators

struct RandomNGs
    rng::StableRNGs.LehmerRNG
    interarrival_time::Function
    service_time::Function
    breakdown_time::Function
    repair_time::Function
end

# constructor functions for RNGs to create all the distributions required

function RandomNGs( P::Parameters )
    rng = StableRNG(P.seed)
    interarrival_time() = rand(rng, Exponential(P.mean_interarrival_time))
    service_time() = (P.mean_service_time)
    breakdown_time() = rand(rng, Exponential(P.mean_breakdown_interval))
    repair_time() = rand(rng, Exponential(P.mean_repair_time))
    
    return RandomNGs( rng, interarrival_time,  service_time, breakdown_time, repair_time )
end


function update!( system::SystemState, e::Event )
    throw( DomainError("invalid event type" ) )
end

# if machine is full
function servers_full(system::SystemState)
    return !isempty(system.in_service)
end

# create initial state structure

function initialise(P::Parameters)
    R = RandomNGs(P) 
    system = SystemState(P) 
    t0 = 0.0
    system.n_events += 1 
    enqueue!(system.event_queue, Arrival(0, t0), t0)
    t1 = 150.0   # add a breakdown at time 150.0                         
    system.n_events += 1
    enqueue!(system.event_queue, Breakdown_start(system.n_events, t1), t1)
    return (system, R)
end

# output function to write state information
function write_state(event_file::IO, system::SystemState, event::Event)
    event_type = typeof(event)
    queue_length = length(system.lawnmower_queue)  # Number of lawnmowers waiting in queue
    in_service_count = length(system.in_service)  # Number of lawnmowers currently being serviced
    machine_status = system.machine_status
    machine_state = machine_status < 1 ? "Online" : "Breakdown"
    repair_time = machine_status < 1 ? "N/A" : string(system.current_repair_time[])
    
    println(event_file, "$(system.time),$(event.id),$event_type,$queue_length,$in_service_count,$machine_status,$machine_state,$repair_time")
end

# output function to write entity information
function write_entity(entity_file::IO, lawnmower::Lawnmower)
    println(entity_file, "$(lawnmower.id),$(lawnmower.arrival_time),$(lawnmower.start_service_time),$(lawnmower.completion_time )")
end


### Update functions


# This function is responsible for moving a lawnmower to machine

function move_to_machine!(system::SystemState, R::RandomNGs)
    if !isempty(system.lawnmower_queue) && system.machine_status == 0
        lawnmower = dequeue!(system.lawnmower_queue)
        lawnmower.start_service_time = system.time
        lawnmower.completion_time = lawnmower.start_service_time + R.service_time()

        enqueue!(system.in_service, lawnmower, lawnmower.completion_time)

        # Schedule a departure event only if the machine is online
        if system.machine_status == 0
            system.n_events += 1
            departure_event = Departure(system.n_events, lawnmower.completion_time)
            enqueue!(system.event_queue, departure_event, lawnmower.completion_time)
        end
    end
end

function update!( system::SystemState, R::RandomNGs, event::Arrival )
    # create an arriving Lawnmower and add it to the  queue
    system.n_entities += 1    # new entity will enter the system
    new_Lawnmower = Lawnmower( system.n_entities, event.time )
    enqueue!(system.lawnmower_queue, new_Lawnmower)
    
    # generate next arrival and add it to the event queue
    future_arrival = Arrival(system.n_events, system.time + R.interarrival_time())
    enqueue!(system.event_queue, future_arrival, future_arrival.time)

    # if the construction machine is available, the Lawnmower goes to service
    if isempty(system.in_service)
        move_to_machine!(system, R)
    end
    return nothing
end


function update!(system::SystemState, R::RandomNGs, event::Breakdown_start)
    system.time = event.time
    system.machine_status = 1  # Machine breaks down
    local repair_time = R.repair_time()
    system.current_repair_time = repair_time

    # Extend completion times for all lawnmowers in service and reschedule their departure events
    for lawnmower in keys(system.in_service)
        old_completion_time = system.in_service[lawnmower]
        new_completion_time = old_completion_time + repair_time
        system.in_service[lawnmower] = new_completion_time
        system.n_events += 1
        new_departure_event = Departure(system.n_events, new_completion_time)
        enqueue!(system.event_queue, new_departure_event, new_completion_time)
    end
    # Reschedule departure events already in the queue that occur during the breakdown
    for (evt, time) in system.event_queue
        if isa(evt, Departure) && time < system.time + repair_time
            dequeue!(system.event_queue, evt)
            rescheduled_time = time + repair_time
            enqueue!(system.event_queue, evt, rescheduled_time)
        end
    end
    
    # Schedule the end of the breakdown using the stored repair time
    system.n_events += 1
    repair_event_time = system.time + repair_time
    enqueue!(system.event_queue, Breakdown_end(system.n_events, repair_event_time), repair_event_time)
end

function update!(system::SystemState, R::RandomNGs, event::Breakdown_end)
    system.time = event.time
    system.machine_status = 0  # Machine is back online

    # Schedule the next breakdown event
    system.n_events += 1
    next_breakdown_time = system.time + R.breakdown_time()
    enqueue!(system.event_queue, Breakdown_start(system.n_events, next_breakdown_time), next_breakdown_time)

    if !isempty(system.lawnmower_queue) # if someone is waiting, move them to service
    system.time = event.time    
    move_to_machine!( system, R )
    end
end


function update!(system::SystemState, R::RandomNGs, event::Departure)
    # If the machine is under repair, reschedule the departure event
    if system.machine_status == 1
        rescheduled_departure_time = event.time + system.current_repair_time
        enqueue!(system.event_queue, Departure(event.id, rescheduled_departure_time), rescheduled_departure_time)
        return nothing
    end

    system.time = event.time

    # There should be a lawnmower in service since the queue is not empty
    if !isempty(system.in_service)
        departing_lawnmower = dequeue!(system.in_service)

        # If there's a lawnmower waiting, move it to service
        if !isempty(system.lawnmower_queue)
            move_to_machine!(system, R)
        end

        return departing_lawnmower
    end
    return nothing
end


### RUN function

function run!(system::SystemState, P::Parameters, R::RandomNGs, fid_state::IO, fid_entities::IO; output_level::Integer=2)
    while system.time < P.T
        if P.seed == 1 && system.time <= 1000.0
            println("$(system.time): ") # Debug information for first few events when seed = 1
        end

        # Grab the next event from the event queue
        event = dequeue!(system.event_queue)
        system.time = event.time  # Update system time to the time of the event
        lawnmower = update!(system, R, event)  # Process the event

        # Write out event and state data
        if output_level >= 2
            write_state(fid_state, system, event)  # Write state information
            if isa(lawnmower, Lawnmower)
                write_entity(fid_entities, lawnmower)  # Write lawnmower information
            end
        end
        system.n_events += 1
    end
    return system
end

# inititialise
seed = 1
T = 1000.00
mean_interarrival_time = 60 
mean_service_time = 45
mean_breakdown_interval = 2880
mean_repair_time = 180
time_units = "minutes"

P = Parameters( seed, T, mean_interarrival_time, mean_service_time,
    mean_breakdown_interval,mean_repair_time, time_units)

# file directory and name; * concatenates strings.
dir = pwd()*"/data/"*"/seed"*string(P.seed) # directory name
mkpath(dir)                          # this creates the directory 
file_entities = dir*"/entities.csv"  # the name of the data file (informative) 
file_state = dir*"/state.csv"        # the name of the data file (informative) 
fid_entities = open(file_entities, "w") # open the file for writing
fid_state = open(file_state, "w")       # open the file for writing

write_metadata( fid_entities )
write_metadata( fid_state )
write_parameters( fid_entities, P )
write_parameters( fid_state, P )

# headers

# headers
println(fid_state, "time,event_id,event_type,queue_length,in_service,machine_status,machine_state,interrupt")
println(fid_entities,"lawnmower.id,lawnmower.arrival_time,lawnmower.start_service_time,lawnmower.completion_time")

# run the actual simulation
(system,R) = initialise( P ) 
run!( system, P, R, fid_state, fid_entities)

# remember to close the files
close( fid_entities )
close( fid_state )




0.0: 
0.0: 
40.49940643026974: 
45.0: 
51.69359364153775: 
90.0: 
117.2350182717608: 
135.0: 
150.0: 
182.98666971174194: 
209.09646771827397: 
378.3393017402848: 
449.01629628507334: 
450.5289898619322: 
471.8491930468444: 
494.567756899112: 
515.3493351445701: 
643.667840213669: 
665.4803608188486: 
666.4093487705566: 
722.6450730711193: 
730.5865405812849: 
180.0: 
225.0: 
270.0: 
315.0: 
360.0: 
405.0: 
450.0: 
495.0: 
540.0: 
585.0: 
630.0: 
675.0: 
760.5865405812849: 
764.1816717017966: 
775.5865405812849: 
784.1905607919427: 
809.1816717017966: 
813.2073074477316: 
829.1905607919427: 
850.96316799232: 
858.2073074477316: 
895.96316799232: 
933.2845552370934: 
978.2845552370934: 
983.7667243883991: 
